# Goat City: Vaccination Status Prediction


## Business Problem:
- Goat city is ordering Covid vaccines and is fully aware that not everyone will get the vaccine. Goat city wants to know how many Covid vaccines they should be ordering. We want to show them that using a survey to predict if a person will get a vaccine is a valid way of estimating how many vaccines to get. 

#### Dataset: Data sourced from DataDriven Datadriven description of the dataset says:

>Vaccines for H1N1 were first publicly available in the United States in October 2009, when the United States government began a vaccination campaign. We will look at data from the National 2009 H1N1 Flu Survey collected to monitor vaccination rates during that campaign. This phone survey asked people whether they had received H1N1 and seasonal flu vaccines, in conjunction with information they shared about their lives, opinions, and behaviors. A better understanding of how these characteristics have been associated with personal vaccination patterns may provide guidance for future public health efforts.
>
The data has already been split into a train and test set, however, we do not have access to the testing set's labels. For now, our group will be focusing ONLY on the h1n1 vaccine label.

- Stakeholder
    - Goat City Government/ Health Department
-Target: H1N1
- Cost of different errors FP/FN
- False Positive: Model predicts they will get the vaccine, but didn't.
Ordering too many vaccines and wasting money/material
- False Negative: Model predicts they won't get the vaccine, but did.
Vaccine shortage. Loss of life.
-Metric:
    - Recall
    - F1-score

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.pipeline import Pipeline
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
import category_encoders as ce

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.metrics import accuracy_score, recall_score, f1_score, roc_auc_score, plot_confusion_matrix

from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
!pip install catboost
from catboost import CatBoostClassifier, Pool

import warnings
warnings.filterwarnings(action='ignore')

from sklearn.metrics import roc_curve, roc_auc_score

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
!pip install category_encoders

In [ ]:
# Training labels
training_labels = pd.read_csv('../Data/training_set_labels.csv', index_col='respondent_id')
training_features = pd.read_csv('../Data/training_set_features.csv', index_col='respondent_id')

## Exploratory Data Analysis

In [1]:
# Training labels
features_df = pd.read_csv('../Data/training_set_features.csv')
lables_df = pd.read_csv('../Data/training_set_labels.csv')

NameError: name 'pd' is not defined

- Lets look into the details of each dataset. 
- The features_df and training_labels will be X within our models.
- The lables_df and training_features will be y within our models.

In [ ]:
# Visually confirm expected results
training_features

- We need to drop the 'seasonal_vaccine' column since our target will be 'h1n1_vaccine.' (This will be done below during data preparation.)

In [ ]:
# Checking features statistics
training_features.describe()

- It appears that ALL of these are categorical variables/features, because there are no "true" floats.

In [ ]:
# Checking Data Types
training_features.dtypes

- We are most likley going to have to OneHotEncode most of these features.

#### Checking for NaN Values

In [ ]:
# Checking Features
training_features.isna().sum()

In [ ]:
# Checking labels
training_labels.isna().sum()

## Data Exploration and Preparation

Describe and justify the process for preparing the data for analysis.

Questions to consider:

Were there variables you dropped or created?
How did you address missing values or outliers?
Why are these choices appropriate given the data and the business problem?
Can you pipeline your preparation steps to use them consistently in the modeling process?

- Our X will be all the variables in features_df. y will be the 'h1n1 vaccine' survey data from lables_df.

In [ ]:
X = features_df
y = lables_df[['h1n1_vaccine']] #we are dropping the "seasonal_vaccine" column

In [ ]:
#X.drop(columns=['respondent_id'], inplace= True)
'''should we leave respondent id in there since it has meaning to Goat City Health Dept.?'''

In [ ]:
X['health_insurance'].value_counts()

In [ ]:
X['health_insurance'].isna().sum()

In [ ]:
X['health_insurance'].fillna(2)

In [ ]:
X['employment_industry'].isna().sum()

In [ ]:
X['employment_occupation'].isna().sum()

- Two columns cannot be just filled in with random object. Therefore, we will work on those columns using CountEncoder and SimpleImputer, which will be covered in next section.

In [ ]:
X['employment_industry'].value_counts()

In [ ]:
X['employment_occupation'].value_counts()

### Preproccessing Pipeline
For the preproccessing, all of the columns are categorical, however, some of them are numerical, and some of them are strings. We will want to handle these these columns differently when imputing missing values.

- Numerical Categories
    - Use Sklearn's Iterative Imputer to fill in the missing values
- String Categories
    - Fill missing values with a new value: 'unknown'
    - One hot encode the results
- Categories with more then 10 unique categories
- We will frequency code these instead, so we don't have an overwhelming amount of columns in the dataframe.

The following process was obtained from following source, as cited:
- Berlin L. Lindseyberlin/Cat-in-the-dat-project. GitHub. https://github.com/lindseyberlin/Cat-in-the-Dat-Project. Published October 17, 2021.

- We will split the data into use set and hold set. Use set will be the one we will use to train and validate the model. Hold set will be our technical test set.

In [ ]:
# split use and hold 
X_use, X_hold, y_use, y_hold = train_test_split(X, y, test_size=0.1, random_state=0)

# split train and val
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)
X_test = X_val #

In [ ]:
# initialize three columns
num_cols = []
ohe_cols = []
freq_cols = []

In [ ]:
# make the lists of columns
# num = any columns with numerical value
# ohe = any columns with object value with less than 10 unique values
# freq = any columns with object value with 10 or more unique values
for c in X.columns:
    if X[c].dtype in ['float64', 'int64']:
        num_cols.append(c)
    elif X[c].nunique() < 10:
        ohe_cols.append(c)
    else:
        freq_cols.append(c)

- Let's check which columns are in each list. (sanity check)

In [ ]:
num_cols

In [ ]:
ohe_cols

In [ ]:
freq_cols

We will create pipeline for each types of cols for preprocessing.

- num: scale with MinMaxScaler and apply IterativeImputer to fill the NA values.
- ohe: apply SimpleImputer to fill NA values and encode with OneHotEncoder.
- freq: encode with CountEncoder and apply SimpleImputer to fill the NA values.

In [ ]:
num_transformer = Pipeline(steps=[
    ('minmaxscaler', MinMaxScaler()),
    ('num_imputer', IterativeImputer(max_iter = 15))
    ])

ohe_transformer = Pipeline(steps=[
    ('ohe_imputer', SimpleImputer(strategy='constant', fill_value='Unknown')),
    ('oh_encoder', OneHotEncoder(handle_unknown='ignore'))
])

freq_transformer = Pipeline(steps=[
    ('freq_encoder', ce.count.CountEncoder(normalize=True, min_group_size=.05)),
    ('freq_imputer', SimpleImputer(strategy='constant', fill_value=0))
])

- Combine 3 pipelines using ColumnTransformer to create 1 preprocessor. Then, fit the preprocessor to X_train dataset.

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_cols),
        ('ohe', ohe_transformer, ohe_cols),
        ('freq', freq_transformer, freq_cols)
    ])

In [ ]:
preprocessor.fit(X_train)

# Modeling
Describe and justify the process for analyzing or modeling the data.

Questions to consider:

How will you analyze the data to arrive at an initial approach?
How will you iterate on your initial approach to make it better?
What model type is most appropriate, given the data and the business problem?

## Modeless Baseline

In [ ]:
# Modeless Baseline
training_labels['h1n1_vaccine'].value_counts(normalize=True)

- In this context, a modeless baseline would have an accuracy of ~0.79 and would guess 0 every single time.

In [ ]:
# Visual for distribution of people who received the H1N1 Vaccine
fig, ax = plt.subplots(figsize=(10,8))

(training_labels['h1n1_vaccine'].value_counts().plot.bar(title="Respondents who Received H1N1 Vaccine", ax=ax))
ax.set_ylabel("Number of People")
plt.xticks([0, 1], ['Unvaccinated', 'Vaccinated'], rotation=0)

### 1. Logistic Regression
Our first model will be logistic regression. We will start with no parameters

In [ ]:
# Create a pipeline combining preprocessor and classifier
# classifier = LogisticRegression()
lr_clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
])

In [ ]:
# Cross validate the model
cross_validate(lr_clf, X_train, y_train, return_train_score=True)

- Cross validation shows that there is no overfitting.

In [ ]:
# fit the model
lr_clf.fit(X_train, y_train)

In [ ]:
# Predict on validation set
lr_preds = lr_clf.predict(X_val)
X_val

- Based on the prediction, we will caculate the metrics: accuracy, recall, f1, and roc_auc.

In [ ]:
print('accuracy: {:0.3f}'.format(accuracy_score(y_val, lr_preds)))
print('recall {:0.3f}'.format(recall_score(y_val, lr_preds)))
print('f1: {:0.3f}'.format(f1_score(y_val, lr_preds)))
print('roc_auc: {:0.3f}'.format(roc_auc_score(y_val , lr_preds)))

- Although the accuracy is high, recall and f1 is less than 0.5. AUC might be at the higher side.

- We will also generate the confusion matrix to visualize the FN/FP cases.

In [ ]:
plot_confusion_matrix(lr_clf, X_val, y_val, cmap = "Blues_r")

### 2. Naive Bayesian
Our second model will be Gaussian Naive Bayesian.

In [ ]:
nb_clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', GaussianNB())
])

In [ ]:
cross_validate(nb_clf, X_train, y_train, return_train_score=True)

- Cross validation shows that there is no overfitting.

In [ ]:
nb_clf.fit(X_train, y_train)

In [ ]:
nb_preds = nb_clf.predict(X_val)

In [ ]:
print('accuracy: {:0.3f}'.format(accuracy_score(y_val, nb_preds)))
print('recall: {:0.3f}'.format(recall_score(y_val, nb_preds)))
print('f1: {:0.3f}'.format(f1_score(y_val, nb_preds)))
print('roc_auc: {:0.3f}'.format(roc_auc_score(y_val , nb_preds)))

- Accuracy score is lower than Logistic Regression. However, recall, f1, ROC_AUC is higher than those of the baseline Logistic Regression Model.

In [ ]:
plot_confusion_matrix(nb_clf, X_val, y_val, cmap="Blues_r")

- Confusion Matrix shows that recall is higher than Precision.

### 3. KNN

Our third model is the K-nearest neighbors.

In [ ]:
knn_clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', KNeighborsClassifier())
])

In [ ]:
knn_clf.fit(X_train, y_train)

In [ ]:
knn_preds = knn_clf.predict(X_val)

In [ ]:
print('accuracy: {:0.3f}'.format(accuracy_score(y_val, knn_preds)))
print('recall: {:0.3f}'.format(recall_score(y_val, knn_preds)))
print('f1: {:0.3f}'.format(f1_score(y_val, knn_preds)))
print('roc_auc: {:0.3f}'.format(roc_auc_score(y_val , knn_preds)))

- Every score seems really low else than accuracy. We will not try to tune this model since the KNN takes a lot of time to run and the model score does not seem promising.

### 4. Random Forest
Our fourth model is Random Forest.

In [ ]:
rf_clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier())
])


In [ ]:
cross_validate(rf_clf, X_train, y_train, return_train_score=True)

- Cross validation score shows that the model is overfitting. The parameter tuning is needed to modify this.

In [ ]:
rf_clf.fit(X_train, y_train)

In [ ]:
rf_preds = rf_clf.predict(X_val)

In [ ]:
print('accuracy: {:0.3f}'.format(accuracy_score(y_val, rf_preds)))
print('recall: {:0.3f}'.format(recall_score(y_val, rf_preds)))
print('f1: {:0.3f}'.format(f1_score(y_val, rf_preds)))
print('roc_auc: {:0.3f}'.format(roc_auc_score(y_val , rf_preds)))

- Most of the score looks high enough. We might want to try out the grid search to modify the scores since the recall is the lowest out of the models we have tried (excluding KNN).

In [ ]:
plot_confusion_matrix(rf_clf, X_val, y_val, cmap="Blues_r")

- As we can see the precision is really high but recall is relatively low.

### 5. Decison Tree
Our fifth model is Decision Tree.

In [ ]:
clf_decision_tree = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier())
])

In [ ]:
cross_validate(clf_decision_tree, X_train, y_train, return_train_score=True)

In [ ]:
clf_decision_tree.fit(X_train, y_train)

In [ ]:
decision_tree_preds = clf_decision_tree.predict(X_val)

In [ ]:
print('accuracy: {:0.3f}'.format(accuracy_score(y_val, decision_tree_preds)))
print('recall: {:0.3f}'.format(recall_score(y_val, decision_tree_preds)))
print('f1: {:0.3f}'.format(f1_score(y_val, decision_tree_preds)))
print('roc_auc: {:0.3f}'.format(roc_auc_score(y_val , decision_tree_preds)))

- Due to the low recall score (0.477) and low f1 score (0.476), although a grid search was performed on this model, it is not included in this notebook. (It can be found [here](https://github.com/austint1121/Flatiron_GOATS_Vaccination_Prediction/blob/Danielle/Notebooks/Danielle/Project3Template-Copy1.ipynb).)

### 6. Catboost
Finally, since all of our data is categorical we will use catboost to create our sixth final model.

In [ ]:
def metrics(y_test, _preds):
    print('accuracy: {:0.3f}'.format(accuracy_score(y_test, _preds)))
    print('recall: {:0.3f}'.format(recall_score(y_test, _preds)))
    print('f1: {:0.3f}'.format(f1_score(y_test, _preds)))
    print('roc_auc: {:0.3f}'.format(roc_auc_score(y_test , _preds)))

In [ ]:
cat_clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('catboost_clf', CatBoostClassifier(task_type='GPU'))
])

In [ ]:
cat_clf.fit(X_train, y_train, verbose=False)

In [ ]:
metrics(y_test, cat_clf.predict(X_test))

- Catboost did about as well as XGboost did, but the biggest thing I noticed is that it took 1/4 of the amount of time to train. I think this would be a much better model type to use going forward.

#### 6.1 Catboost Tuning

In [ ]:
# Create pipleline with tuned params
tuned_cat_clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    # Changing the eval metric from "logloss" to "AUC" and modifying the learning rate
    ('catboost_clf', CatBoostClassifier(learning_rate=0.03,
    eval_metric='AUC', task_type='GPU'))
])

In [ ]:
# Fit the new model
tuned_cat_clf.fit(X_train, y_train, verbose=False)

In [ ]:
# Calculate metrics
metrics(y_test, tuned_cat_clf.predict(X_test))

#### 6.2 More Tuned Catboost

In [ ]:
tuned_cat_clf2 = Pipeline(steps=[
    ('preprocessor', preprocessor),
    # For these parameters, I selected many common hypertuning parameters and gave them a generic value
    ('catboost_clf2', CatBoostClassifier(eval_metric='AUC', task_type='GPU', iterations=500,
                                         random_strength=5,
                                         bagging_temperature=5,
                                         max_bin=5,
                                         grow_policy='Lossguide',
                                         min_data_in_leaf=5,
                                         max_depth=5,
                                         l2_leaf_reg=10,
                                         auto_class_weights='Balanced'))
])

In [ ]:
tuned_cat_clf2.fit(X_train, y_train, verbose=False)

In [ ]:
metrics(y_test, tuned_cat_clf2.predict(X_test))

- Conclusion:
Just from manually adding some parameters and tuning them by hand, we have a significant increase to our metrics.

#### 6.3 Catboost Tuning: Min Trees/Max_leaves

In [ ]:
tuned_cat_clf3 = Pipeline(steps=[
    ('preprocessor', preprocessor),
    # For these parameters, I selected many common hypertuning parameters and gave them a generic value
    ('catboost_clf3', CatBoostClassifier(eval_metric='AUC', task_type='GPU', iterations=500,
                                         loss_function='Logloss',
                                         random_strength=4,
                                         bagging_temperature=3,
                                         max_bin=5,
                                         grow_policy='Lossguide',
                                         min_data_in_leaf=5,
                                         max_depth=5,
                                         l2_leaf_reg=300,
                                         auto_class_weights='Balanced',
                                         best_model_min_trees=3,
                                         max_leaves=30
                                         ))
])

In [ ]:
tuned_cat_clf3.fit(X_train, y_train, verbose=False)

In [ ]:
metrics(y_test, tuned_cat_clf3.predict(X_test))

Evaluation
The evaluation of each model should accompany the creation of each model, and you should be sure to evaluate your models consistently.

Evaluate how well your work solves the stated business problem.

Questions to consider:

How do you interpret the results?
How well does your model fit your data? How much better is this than your baseline model? Is it over or under fit?
How well does your model/data fit any relevant modeling assumptions?
For the final model, you might also consider:

How confident are you that your results would generalize beyond the data you have?
How confident are you that this model would benefit the business if put into use?
What does this final model tell you about the relationship between your inputs and outputs?